# Retrieve UK Biobank lipids phenotypes and covariates

This notebook retrieves lipid phenotypes and covariates from the database using Spark SQL and stores that extract as a CSV for further downstream use.

# Setup

<div class='alert alert-block alert-warning'>
This notebook will run correctly on the UK Biobank Research Analysis Platform.
</div>

In [ ]:
import dxdata
import pandas as pd
import re

In [ ]:
# Initialize dxdata engine
engine = dxdata.connect(dialect='hive+pyspark')
pt = engine.execute('SET spark.sql.shuffle.partitions=50').to_pandas()

In [ ]:
dataset = dxdata.load_dataset('app7089_202103231620.dataset')

In [ ]:
participant = dataset['participant']

## Discover the fields of interest 

In [ ]:
def print_field_list(fields):
    for field in sorted(fields, key=lambda fld: fld.title):
        print(f'\n{field.column_name}: {field.title}')
        print(f'\t{field.units}')
        print(f'\t{field.type}')
        print(f'\t{field.coding}')
        if field.coding is not None and field.coding.name != 'data_coding_4':
            print(f'\t{field.coding.codes}')

In [ ]:
fields_by_title = list(participant.find_fields(titles=['Sex', 'Date of birth']))
print_field_list(fields_by_title)

### Where is field 'Date of Birth'? 

In [ ]:
print_field_list(list(participant.find_fields(names=['p33', 'p31'])))

In [ ]:
print_field_list(list(participant.find_fields(name_regex='(?i)p33')))

In [ ]:
print_field_list(list(participant.find_fields(title_regex='(?i)birth')))

In [ ]:
fields_by_title_regex = list(participant.find_fields(title_regex='(?i)cholesterol|hdl|ldl|triglycerides|Age when attended assessment centre|treatment/medication code'))
len(fields_by_title_regex)

In [ ]:
all_fields = fields_by_title + fields_by_title_regex
print_field_list(all_fields)

In [ ]:
drug_mapping = {k : field.coding.codes[str(k)] for field in all_fields
    if field.coding is not None and field.coding.name == 'data_coding_4'
        for k in [1140861958, 1140861970, 1140864592, 1140881748, 1140888594, 1140888648, 1140910632, 1140910654, 1141146138, 1141146234, 1141192410, 1141192414, 1141200040]}

drug_mapping

In [ ]:
drug_mapping_df = pd.DataFrame.from_dict(drug_mapping, orient='index', columns=['drug_name']).rename_axis('drug_number').reset_index()

drug_mapping_df

## Retrieve the data 

In [ ]:
import time

start = time.time()
pheno_data = participant.retrieve_fields(engine=engine, fields=all_fields, coding_values='replace').toPandas()
end = time.time()
print(end - start)

In [ ]:
pheno_data.shape

In [ ]:
# Uncomment to see row level data.
#pheno_data.head()

In [ ]:
pheno_data.columns

### Construct improved column names 

In [ ]:
col_names = {'eid': 'eid'}
for field in sorted(all_fields, key=lambda fld: fld.name):
    name = '_'.join([field.column_name, re.sub(' \| Instance \d', '', field.title).replace(' ', '_').replace('/', '_')])
    if field.units is not None:
        name += f'_{field.units.replace(' ', '_').replace("/", "_")}'
    print(name)
    col_names[field.column_name] = name

In [ ]:
pheno_data = pheno_data.rename(columns=col_names)

In [ ]:
pheno_data.columns

## Write out the data extract to a CSV 

In [ ]:
drug_mapping_df.to_csv('drug_mapping.csv', index=False)

In [ ]:
%%bash

dx upload drug_mapping.csv

In [ ]:
pheno_data.to_csv('lipids.csv', index=False)

In [ ]:
%%bash

dx upload lipids.csv

# Provanance

In [ ]:
import datetime
print(datetime.datetime.now())

In [ ]:
!pip3 freeze